# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:
# ID: 336490347
# Anton Iosifov

# Details Student 2:
# ID: 336490271
# Valentin Gundorov


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for hebrew_tokenizer: filename=hebrew_tokenizer-2.3.0-py3-none-any.whl size=13418 sha256=60d332f93bfb20696e4209bfe2226d5e0128f8e59f700ccdf8d061270cc33c03
  Stored in directory: c:\users\gundo\appdata\local\pip\cache\wheels\a7\2f\af\95d5eb2e7c291753542d4419e200429a22fb2d74e3f27d7711
Successfully built hebrew_tokenizer


In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\gundo


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(20)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f
8,עד לפני כמה חודשים אני הייתי בטוח שכל חיי אהיה...,m
9,"בן זוגי חגג יום הולדת, ורציתי להפתיע אותו ולסג...",f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [11]:
def tokenization(text):
    token = ht.tokenize(text)                                      #token = [('Hebrew,aba'), ('English',home), ....]
    tokens_list = list(filter(lambda x: (x[0] == 'HEBREW'),token)) #token_list = [('Hebrew',aba), ('Hebrew',ima)....]
    tokens_list = [tuple[1] for tuple in tokens_list]              #token_list = [(aba), (ima)....]
    return tokens_list                                             #return only hebrew tokens list

In [12]:
#----------------------------------Label Encoding-------------------------------------------------------------------------------

df_train["gender"] = df_train["gender"].replace({"m": 1, "f": 0}) #New column 'gender' in df_train, if gender: m = 1, f = 0.

X_train = df_train['story']   #Stories
y_train = df_train['gender']  #genders
X_train.head(5)
y_train.head(20)


0    כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...
1    לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...
2    מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...
3    כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...
4    ‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...
Name: story, dtype: object

0     1
1     1
2     0
3     1
4     0
5     0
6     1
7     0
8     1
9     0
10    1
11    0
12    0
13    1
14    0
15    0
16    1
17    1
18    1
19    1
Name: gender, dtype: int64

In [13]:
#----------------------------------Preprocessing and Feature Extraction---------------------------------------------------------

# LogisticRegression
SGDClassifier_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer = tokenization)),    #Vectorization with TF-IDF method.
    ('normalize', preprocessing.Normalizer(norm = 'l2')),    #Normalizing of vectors
    ('clf', SGDClassifier(max_iter = 1000, random_state=42)) #text data classification using stochastic gradient descent algorithm
])



# KNeighborsClassifier
knn_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer = tokenization)),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('classifier', KNeighborsClassifier())
])



# Naive bayes
NB_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer = tokenization)),
    ('normalize', preprocessing.Normalizer(norm = 'l2')),
    ('clf', MultinomialNB())
])



In [14]:
#----------------------------------Grid Search and Hyperparameter Tuning---------------------------------------------------------
# GridSearchCV is used to perform a grid search with 10-fold cross-validation to find the best combination 
# of hyperparameters that maximize the F1 score on the training data.

best_model = None
best_f1_score = 0

# Definition of parameter grids for each pipeline:

SGDClassifier_param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
    'clf__penalty': ['l1', 'l2', 'elasticnet'],
    'clf__max_iter': [1000, 2000, 3000]
}


knn_param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'classifier__n_neighbors': [3, 5, 7, 9],
    'classifier__weights': ['uniform', 'distance'],
    'classifier__p': [1, 2]
}

NB_param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'clf__alpha': [0.0001, 0.001, 0.01, 0.1]
}

# Define the list of pipelines to evaluate
pipelines = [
    ('SGDClassifier', SGDClassifier_pipeline, SGDClassifier_param_grid),
    ('Naive Bayes', NB_pipeline, NB_param_grid),
    ('KNN', knn_pipeline, knn_param_grid)
]


In [15]:
# Best Model Selection
# The best model with the highest cross-validation F1 score is selected and stored in the best_model variable.

# Loop over the pipelines and perform grid search with cross-validation
for name, pipeline, parameter_grid in pipelines:
    print(f"Evaluating {name}...")
    grid_search = GridSearchCV(pipeline, parameter_grid, cv=10, scoring='f1_macro')
    grid_search.fit(X_train, y_train)
    print(f"Best hyperparameters for {name}: {grid_search.best_params_}")
    print("CV score=%0.3f" % grid_search.best_score_)
    if grid_search.best_score_ > best_f1_score:
        best_f1_score = grid_search.best_score_
        best_model_name = name
        best_model = grid_search.best_estimator_

# Print the best model's name and F1 score
print(f"Best model: {best_model_name} with F1: {best_f1_score}.")

Evaluating SGDClassifier...


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function tokenization at 0x000001FD2F472DC0>)),
                                       ('normalize', Normalizer()),
                                       ('clf',
                                        SGDClassifier(random_state=42))]),
             param_grid={'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
                         'clf__max_iter': [1000, 2000, 3000],
                         'clf__penalty': ['l1', 'l2', 'elasticnet'],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters for SGDClassifier: {'clf__alpha': 0.0001, 'clf__max_iter': 1000, 'clf__penalty': 'l1', 'tfidf__max_df': 0.75}
CV score=0.710
Evaluating Naive Bayes...


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function tokenization at 0x000001FD2F472DC0>)),
                                       ('normalize', Normalizer()),
                                       ('clf', MultinomialNB())]),
             param_grid={'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters for Naive Bayes: {'clf__alpha': 0.01, 'tfidf__max_df': 0.5}
CV score=0.508
Evaluating KNN...


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function tokenization at 0x000001FD2F472DC0>)),
                                       ('normalize', Normalizer()),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid={'classifier__n_neighbors': [3, 5, 7, 9],
                         'classifier__p': [1, 2],
                         'classifier__weights': ['uniform', 'distance'],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters for KNN: {'classifier__n_neighbors': 5, 'classifier__p': 2, 'classifier__weights': 'distance', 'tfidf__max_df': 1.0}
CV score=0.608
Best model: SGDClassifier with F1: 0.710115679911672.


In [16]:
#----------------------------------Model improvement----------------------------------------------------------------------------
# The code then refines the hyperparameters for the SGDClassifier model using another round of grid search, 
# aiming to find the best possible hyperparameters for the chosen model.

SGDClassifier_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer = tokenization,max_df=0.65,ngram_range=(1,2))),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('clf', SGDClassifier(random_state=42))
])

# Set up the parameter grid
SGDClassifier_param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
    'clf__penalty': ['l1', 'l2', 'elasticnet'],
    'clf__max_iter': [30, 40, 50],
    'clf__loss': ['hinge', 'log'],
    'clf__validation_fraction': [0.1, 0.2]
}

# Perform grid search
grid_search = GridSearchCV(SGDClassifier_pipeline, SGDClassifier_param_grid, cv=10, scoring='f1_macro')
grid_search.fit(X_train, y_train)

#----------------------------------Best hyperparameters and cross-validation score----------------------------------------------------------------

print("Best hyperparameters:", grid_search.best_params_)
print("Cross-validation F1 score:", grid_search.best_score_)


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_df=0.65,
                                                        ngram_range=(1, 2),
                                                        tokenizer=<function tokenization at 0x000001FD2F472DC0>)),
                                       ('normalize', Normalizer()),
                                       ('clf',
                                        SGDClassifier(random_state=42))]),
             param_grid={'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
                         'clf__loss': ['hinge', 'log'],
                         'clf__max_iter': [30, 40, 50],
                         'clf__penalty': ['l1', 'l2', 'elasticnet'],
                         'clf__validation_fraction': [0.1, 0.2],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters: {'clf__alpha': 0.0001, 'clf__loss': 'hinge', 'clf__max_iter': 30, 'clf__penalty': 'l1', 'clf__validation_fraction': 0.1, 'tfidf__max_df': 1.0}
Cross-validation F1 score: 0.7242193823141414


In [17]:
best_parameters = {
    'tfidf__max_df': 0.75,
    'clf__alpha': 0.0001,
    'clf__penalty': 'l1',
    'clf__max_iter': 30,
    'clf__loss': 'hinge',
    'clf__validation_fraction': 0.1
}

In [18]:
#---------------------------------Vectorization with TFidf-----------------------------------------------------------------------------------------

best_model_tfidf = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer = tokenization, max_df = best_parameters['tfidf__max_df'], ngram_range=(1, 2))),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('clf', SGDClassifier(random_state=42, alpha=best_parameters['clf__alpha'], penalty=best_parameters['clf__penalty'], 
            max_iter=best_parameters['clf__max_iter'], loss=best_parameters['clf__loss'],
            validation_fraction=best_parameters['clf__validation_fraction']))
])


best_model_tfidf.fit(X_train, y_train)

f1_scores = cross_val_score(best_model_tfidf, X_train, y_train, cv=10, scoring='f1_macro')
avg_f1_score = np.mean(f1_scores)
print("F1 score for the best selected model:", avg_f1_score)   #F1 average(model quality check)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.75, ngram_range=(1, 2),
                                 tokenizer=<function tokenization at 0x000001FD2F472DC0>)),
                ('normalize', Normalizer()),
                ('clf',
                 SGDClassifier(max_iter=30, penalty='l1', random_state=42))])

F1 score for the best selected model: 0.7199367968560483


In [19]:
#----------------------------------Vectorization with countvectorizer---------------------------------------------------------------------

best_model_countvectorizer = Pipeline([ ('tfidf', CountVectorizer(tokenizer = tokenization,max_df=0.65, ngram_range=(1, 2))),
                                        ('normalize', preprocessing.Normalizer(norm='l2')),
                                        ('clf', SGDClassifier(max_iter=22, random_state=42, alpha=0.0001, penalty='l1', loss='hinge',learning_rate='optimal', eta0=1, validation_fraction=0.1, n_iter_no_change=6)) ])

best_model_countvectorizer.fit(X_train, y_train)

f1_scores = cross_val_score(best_model_countvectorizer, X_train, y_train, cv=10, scoring='f1_macro')
avg_f1_score = np.mean(f1_scores)
print("Average F1 score:", avg_f1_score) #F1 average(model quality check)

Pipeline(steps=[('tfidf',
                 CountVectorizer(max_df=0.65, ngram_range=(1, 2),
                                 tokenizer=<function tokenization at 0x000001FD2F472DC0>)),
                ('normalize', Normalizer()),
                ('clf',
                 SGDClassifier(eta0=1, max_iter=22, n_iter_no_change=6,
                               penalty='l1', random_state=42))])

Average F1 score: 0.6863719138696902


In [20]:
#----------------------------------Prediction-----------------------------------------------------------------------------------

df_predicted = pd.DataFrame(columns = ['test_example_id','predicted_category'])
df_predicted

y_pred = best_model_tfidf.predict(df_test["story"])
df_predicted['test_example_id'] = df_test['test_example_id']
df_predicted['predicted_category'] = np.where(y_pred == 1, 'm', 'f')
df_predicted.head(20)

,test_example_id,predicted_category


,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f
5,5,m
6,6,m
7,7,f
8,8,m
9,9,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [21]:
df_predicted.to_csv('classification_results.csv',index=False)